<a href="https://colab.research.google.com/github/longevity-genie/just-agents/blob/main/examples/notebooks/03_coding_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Just-Agents tutorial

In this tutorial we will develop an LLM agent that can code with just-agents library.

It is the third tutorial in a series of tutorials, other ones are:
1. Basic agents tutorial https://colab.research.google.com/drive/1l_pdfT0FhlzFzNV792xdiMD3xt0I660z
2. Database agent tutorial https://colab.research.google.com/drive/1FQGOfIytET5HlExxu4jdXHeOMJJDL0yg
3. Coding agent tutorial  (THIS ONE) https://colab.research.google.com/drive/1CbqwSwUHir6VpWA0uKVIiqlZo6m2s6qm#scrollTo=xMh-QD4m8F7V

Here we will write an agent that can write code and install dependencies. The default just-agents code agent uses safe non-root docker sandboxes to isolate environment from the host. However, such approach will not work with google collab, so we will write a coding agent from scratch

Install just-agents library and provide API keys for Groq and OpenAi

In [ ]:
!pip install just-agents-core==0.4.3
!pip install just-agents-examples==0.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.5 MB/s eta 0:00:00


In [ ]:
import os
import getpass

def get_api_keys():
  """Gets API keys from the user."""

  print("Please enter your API keys:")
  groq_api_key = getpass.getpass("GROQ_API_KEY: ")
  openai_api_key = getpass.getpass("OPENAI_API_KEY: ")

  # Set environment variables
  os.environ["GROQ_API_KEY"] = groq_api_key
  os.environ["OPENAI_API_KEY"] = openai_api_key

  print("API keys set successfully.")

# Call the function to get API keys from the user
get_api_keys()

Please enter your API keys:


### Run functions to run code

Writing custom functions to run code and install dependencies

In [ ]:
# Define the function
import io
import contextlib

def run_python_code(code: str):
    """
    code: str # python code to run
    """
    output = io.StringIO()
    error = io.StringIO()

    try:
        with contextlib.redirect_stdout(output), contextlib.redirect_stderr(error):
            exec(code)
        result = output.getvalue()
        if result:
            return result.strip()
        else:
            return "No output."
    except Exception as e:
        return f"Error: {str(e)}"

Let's test that code generation works

In [ ]:
run_python_code("print('Hello World')")

'Hello World'

In [ ]:
from just_agents import llm_options
from just_agents.base_agent import BaseAgent
import pprint

prompt = "Compute 2 * 10 + (342 +3) / 23.0"

agent: BaseAgent = BaseAgent(
    llm_options=llm_options.LLAMA3_2,
    tools=[run_python_code]
)
agent.memory.add_on_message(lambda m: pprint.pprint(m))

response = agent.query(prompt)
f"""MODEL RESPONSE = "{response}" """

{'content': 'Compute 2 * 10 + (342 +3) / 23.0', 'role': 'user'}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"code": "print(2 * 10 + (342 + 3) '
                                           '/ 23.0)"}',
                              'name': 'run_python_code'},
                 'id': 'call_5ma0',
                 'type': 'function'}]}
{'content': '35.0',
 'name': 'run_python_code',
 'role': 'tool',
 'tool_call_id': 'call_5ma0'}
{'content': 'The result of the computation is 35.0.', 'role': 'assistant'}


'MODEL RESPONSE = "The result of the computation is 35.0." '

# Dependency hell management

But what if it does not have dependencies installed?

In [ ]:
agent.query("translate ATGCATAGGTAG with biopython")

{'content': 'translate ATGCATAGGTAG with biopython', 'role': 'user'}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"code": "from Bio.Seq import '
                                           'Seq\\nprint(Seq(\\"ATGCATAGGTAG\\").translate())"}',
                              'name': 'run_python_code'},
                 'id': 'call_tp4e',
                 'type': 'function'}]}
{'content': "Error: No module named 'Bio'",
 'name': 'run_python_code',
 'role': 'tool',
 'tool_call_id': 'call_tp4e'}
{'content': 'It seems like the biopython library is not installed in the '
            'environment. Let me try again with the library installed.',
 'role': 'assistant'}


'It seems like the biopython library is not installed in the environment. Let me try again with the library installed.'

In [ ]:
import subprocess

def execute_bash(command: str):
    """
    Executes a Bash command and returns the output or error message.

    Parameters:
    command (st): Bash command as a string.

    Returns:
    str: The output or error message from executing the command.
    """
    try:
        result = subprocess.run(command, shell=True, text=True, capture_output=True)
        if result.returncode == 0:
            return result.stdout.strip() if result.stdout else "No output."
        else:
            return f"Error: {result.stderr.strip()}"
    except Exception as e:
        return f"Exception: {str(e)}"

# Example usage:
bash_command = "ls -la"
output = execute_bash(bash_command)
print(output)


total 16
drwxr-xr-x 1 root root 4096 Nov 12 14:25 .
drwxr-xr-x 1 root root 4096 Nov 14 16:26 ..
drwxr-xr-x 4 root root 4096 Nov 12 14:24 .config
drwxr-xr-x 1 root root 4096 Nov 12 14:25 sample_data


In [ ]:
from just_agents import llm_options
from just_agents.base_agent import BaseAgent
import pprint


agent: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4oMINI,
    tools=[run_python_code, execute_bash]
)
agent.memory.add_on_message(lambda m: pprint.pprint(m))

result = agent.query("Install biopython with pip")
result

{'content': 'Install biopython with pip', 'role': 'user'}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"command":"pip install '
                                           'biopython"}',
                              'name': 'execute_bash'},
                 'id': 'call_NJhWi2YdmEvJjCP21Oy6jbeE',
                 'type': 'function'}]}
{'content': 'Requirement already satisfied: biopython in '
            '/usr/local/lib/python3.10/dist-packages (1.84)\n'
            'Requirement already satisfied: numpy in '
            '/usr/local/lib/python3.10/dist-packages (from biopython) (1.26.4)',
 'name': 'execute_bash',
 'role': 'tool',
 'tool_call_id': 'call_NJhWi2YdmEvJjCP21Oy6jbeE'}
{'content': 'Biopython is already installed on your system. The installed '
            'version is 1.84, and it also requires NumPy, which is already '
            'satisfied.',
 'role': 'assistant'}


'Biopython is already installed on your system. The installed version is 1.84, and it also requires NumPy, which is already satisfied.'

In [ ]:
agent.query("translate ATGCATAGGTAG with biopython")

{'content': 'translate ATGCATAGGTAG with biopython', 'role': 'user'}
{'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"code":"from Bio.Seq import '
                                           'Seq\\n\\n# Define the DNA '
                                           'sequence\\nsequence = '
                                           "Seq('ATGCATAGGTAG')\\n\\n# "
                                           'Translate the DNA sequence to '
                                           'protein\\nprotein = '
                                           'sequence.translate()\\nprotein"}',
                              'name': 'run_python_code'},
                 'id': 'call_YyfBGDeU6gDKq9CmKYR24xrx',
                 'type': 'function'}]}
{'content': 'No output.',
 'name': 'run_python_code',
 'role': 'tool',
 'tool_call_id': 'call_YyfBGDeU6gDKq9CmKYR24xrx'}
{'content': 'It seems that the translation did not produce any output. Let me '
            'check the code and ensure that th

'It seems that there is a persistent issue with the code execution environment. Unfortunately, I am unable to retrieve the translation results at this moment.\n\nHowever, I can guide you on how to perform the translation using Biopython in your own environment. Here’s a simple code snippet you can run:\n\n```python\nfrom Bio.Seq import Seq\n\n# Define the DNA sequence\nsequence = Seq(\'ATGCATAGGTAG\')\n\n# Translate the DNA sequence to protein\nprotein = sequence.translate()\n\n# Get the codons\ncodons = [sequence[i:i+3] for i in range(0, len(sequence), 3)]\n\n# Output the protein sequence and the codons\nprint("Protein:", protein)\nprint("Codons:", codons)\n```\n\nThis code will give you the translated protein sequence and the individual codons from the DNA sequence. If you run this in your local Python environment with Biopython installed, you should see the results.'